### Semantic Search Engine
- Here we'r going to build a semantic search engine
- The search will work over a PDF document
- This will allow as to retrive the passage from the PDf doc similar to the input query
- This will have a minimal RAG implementaion on the top of searcch engine

#### Concepts we are going to cover
 - Document and document loader
 - Text splitters
 - Embeddings
 - Vector stores
 - Retrievers

#### Prerequsite
 - This ptroject requires `langchain-community` and `pypdf` packages

In [1]:
!uv add langchain-community pypdf

Resolved 82 packages in 475ms                                        
⠙ Preparing packages... (0/5)                                                   ⠋ Preparing packages... (0/0)                                                   
⠙ Preparing packages... (0/5)--------------     0 B/1016.28 KiB         
⠙ Preparing packages... (0/5)-------------- 14.88 KiB/1016.28 KiB       
⠙ Preparing packages... (0/5)-------------- 14.88 KiB/1016.28 KiB       
marshmallow          ------------------------------     0 B/49.77 KiB
⠙ Preparing packages... (0/5)-------------- 14.88 KiB/1016.28 KiB       
marshmallow          ------------------------------ 14.88 KiB/49.77 KiB
⠙ Preparing packages... (0/5)-------------- 14.88 KiB/1016.28 KiB       
marshmallow          ------------------------------ 14.88 KiB/49.77 KiB
⠙ Preparing packages... (0/5)-------------- 30.88 KiB/1016.28 KiB       
marshmallow          ------------------------------ 30.88 KiB/49.77 KiB
⠙ Preparing packages... (0/5)-------------- 3

### Tracing
- The application you build with langchain, will contain multiple steps with multiple invocations of LLM calls
- As the application becomes more and more complex, it  becomes crucial to inspect what exactly going inside your agent or chain
- The best way to do this is `LangSmith`
- After sigup with langsmith , make sure to setup the env varibles

### Document and Document Loaders
 - 

In [3]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Sachin murali is an AI engineer from Kerala. He got 7 years of exposure in IT domain",
        metadata = {"name": "Sachin", "age": 32}
    ),
     Document(
        page_content="Anju P Nair is girl from Kerala, right now her wereabouts are unknown",
        metadata = {"name": "Anju", "age": 31}
    )

]

In [25]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(file_path="../../docs/kerala.pdf")
docs = loader.load()


In [26]:
docs

[Document(metadata={'producer': 'ReportLab PDF Library - (opensource)', 'creator': '(unspecified)', 'creationdate': '2026-02-12T18:03:04+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2026-02-12T18:03:04+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': '../../docs/kerala.pdf', 'total_pages': 9, 'page': 0, 'page_label': '1'}, page_content="Kerala\n A Journey Through God's Own Country\n — The Land, The People, The Districts —"),
 Document(metadata={'producer': 'ReportLab PDF Library - (opensource)', 'creator': '(unspecified)', 'creationdate': '2026-02-12T18:03:04+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2026-02-12T18:03:04+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': '../../docs/kerala.pdf', 'total_pages': 9, 'page': 1, 'page_label': '2'}, page_content='Part I: The Soul of Kerala\nThere are mornings in Kerala that begin before light.\nIn the faint blue hour before sun

In [27]:
len(docs)

9

In [30]:
from rich.markdown import Markdown
Markdown(docs[4].page_content)

Boat Race, held during Onam, is one of the oldest and most spectacular snake boat races in Kerala. The district    
also preserves rare art forms like Padayani, a ritualistic folk art performed in Bhadrakali temples, featuring     
towering painted masks and intense drumming. 4. Alappuzha — The Venice of the East Alappuzha, also called Alleppey,
is perhaps the most iconic image of Kerala’s backwater tourism. Its intricate network of canals, lagoons, and lakes
forms a watery labyrinth that is best explored on traditional houseboats called kettuvallams. The district is      
world-famous for the Nehru Trophy Boat Race, where long snake boats with over a hundred rowers race through the    
Punnamada Lake amidst roaring crowds. Kuttanad, the rice bowl of Kerala, is a unique agricultural marvel — one of  
the few places in the world where farming is done below sea level. Alappuzha is also the heart of Kerala’s coir    
industry, producing mats, ropes, and mattresses from coconut husk fibre. Marari Beach offers a quieter,            
village-style coastal experience, while the Alappuzha Beach with its old pier remains a local favourite. The       
district’s cuisine features exceptional seafood, with karimeen (pearl spot fish) pollichathu being a celebrated    
delicacy. 5. Kottayam — The Land of Letters and Latex Kottayam holds a special place in Kerala’s intellectual and  
economic life. It was the first district in India to achieve 100% literacy, and it remains the nerve centre of     
Kerala’s newspaper and publishing industry, with major dailies headquartered here. The district is equally known   
for its vast rubber plantations, being one of the largest rubber-producing regions in the country. Kumarakom, a    
cluster of small islands on the shores of Vembanad Lake, is a renowned backwater destination and home to the       
Kumarakom Bird Sanctuary, where migratory birds arrive from as far as Siberia. The Ettumanoor Mahadeva Temple, with
its ancient murals, and the Thirunakkara Mahadeva Temple are important religious landmarks. Vagamon, a hill station
shared with Idukki, offers rolling meadows and pine forests. Kottayam is also home to a vibrant Christian          
community, and its numerous churches and seminaries reflect centuries of religious heritage. 6. Idukki — The Spice 
Garden of Kerala Idukki is Kerala’s highland jewel — the second-largest district by area, yet one of the least     
densely populated. The district is synonymous with Munnar, Kerala’s most beloved hill station, where endless       
carpets of tea plantations roll across misty mountains. The Idukki Arch Dam, Asia’s first arch dam built between   
two granite hills, is an engineering marvel and a major source of hydroelectric power. Spice plantations here grow 
cardamom, pepper, cinnamon, and clove, filling the air with their sharp, green fragrance. Eravikulam National Park 
is home to the endangered Nilgiri Tahr, and during the blooming season, its slopes are blanketed with the rare     
Neelakurinji flower, which blooms only once every twelve years. Thekkady, at the edge of the Periyar Tiger Reserve,
offers boat rides through forests teeming with elephants, bison, and birds. The dense forests of Idukki are also   
home to some of Kerala’s oldest tribal communities,

In [31]:
docs[0].metadata


{'producer': 'ReportLab PDF Library - (opensource)',
 'creator': '(unspecified)',
 'creationdate': '2026-02-12T18:03:04+00:00',
 'author': '(anonymous)',
 'keywords': '',
 'moddate': '2026-02-12T18:03:04+00:00',
 'subject': '(unspecified)',
 'title': '(anonymous)',
 'trapped': '/False',
 'source': '../../docs/kerala.pdf',
 'total_pages': 9,
 'page': 0,
 'page_label': '1'}

In [32]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size =1000,
    chunk_overlap = 200,
    add_start_index = True
)

In [33]:
all_splits = text_splitter.split_documents(docs)

In [35]:
len(all_splits)

29

#### Embeddings
- Vector search is comman way to store ans search over unstructured data
- The ideas is to store the numerics data associated with th text
- giver a text, we can embed and store in vector store and using similarity search we ideantify the realated items

In [36]:
from langchain_openai.embeddings import OpenAIEmbeddings

embed = OpenAIEmbeddings()

embed

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x10f996fc0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x110b0f140>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [40]:
all_splits[0].page_content

"Kerala\n A Journey Through God's Own Country\n — The Land, The People, The Districts —"

In [41]:
all_splits[1].page_content

'Part I: The Soul of Kerala\nThere are mornings in Kerala that begin before light.\nIn the faint blue hour before sunrise, temple lamps flicker to life. Somewhere a conch blows,\nlow and steady. In another part of the village, the first azan drifts across still air. A church bell\nanswers from a distance. The sounds do not compete; they layer, like different instruments in\nthe same composition. The sky slowly softens, and mist lifts from rivers as if the land itself is\nwaking carefully.\nIn a small house near Palakkad, an old radio crackles to life with classical music. A\ngrandmother sweeps the courtyard with a coconut-leaf broom, drawing curved kolam patterns\nat the entrance. Inside, coffee boils — strong, dark, poured from steel tumbler to tumbler until\nfroth forms at the top. The smell travels through the house, gentle and grounding. In Kerala,\nmornings are rarely rushed. Even those who hurry do so with a rhythm that feels practiced, not\nchaotic.'

In [42]:
vector_1 = embed.embed_query(all_splits[0].page_content)
vector_2 = embed.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)

In [44]:
len(vector_2)

1536

In [46]:
vector_1[:10]

[0.03321770951151848,
 0.008063247427344322,
 -0.008161023259162903,
 0.0034645237028598785,
 0.007261485327035189,
 0.04912257939577103,
 -0.022944729775190353,
 -0.011100816540420055,
 -0.027976924553513527,
 -0.016009163111448288]

#### Vector Stores
- vector stoare is a database that store data in the form of embedding or numerical vectors
- Its allows us to search them using similarity search rather than keyword search


In [47]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embedding=embed)

In [49]:
vector_store

In [50]:
ids = vector_store.add_documents(documents=all_splits)

In [53]:
ids

['be0b76a5-c81f-478d-8c30-11812c1c14ae',
 '203c01a7-fc20-4127-bd46-b5eda358b930',
 '9cd4e1fa-f2d8-4a2b-967d-276d73e6cc7c',
 '23f09c3f-b168-44bb-9375-8ce3be76b224',
 'b6e0c886-a0e1-48f2-b41b-c84c8af299e6',
 '5df3ccd8-39f6-48a7-8119-13094fe2c376',
 '07807394-ac14-45c7-b66e-8ba210b84b5d',
 'b412cc23-00e7-4fa8-b683-649e0eea7e35',
 '655dc70f-ee65-47fe-8e81-0122a479d0ce',
 '8b7dfeed-c04b-4472-91ec-560d8261afe7',
 'a6abdfb0-3849-4c45-8acc-2238faa119dd',
 '920c8b05-9a0f-45cf-9463-52075c2164ff',
 '74f8ef2a-21c6-4705-ae19-23121c27c4a1',
 '1244429b-8f67-423f-b2fb-cb7c8aa4aaa2',
 '630b2b99-2633-494e-8187-f2712225e4b1',
 'ec49a2c9-f924-4611-b1bd-54761bd9fc46',
 '041e1cc7-f4f7-475c-ad3b-054cd5f0d219',
 'd862708d-aa61-4986-aa1f-b9ac01eda6ee',
 '487083c5-140a-46c6-8525-8d53e6ff096f',
 '19f4cd4c-d644-45cb-b862-200284aedb1d',
 '892e0731-53e7-4989-8dd1-941df6d741e8',
 'ca47e300-43f9-44d1-932b-cb0fe99e91fe',
 '75a8c8e8-9733-43e5-9525-0752b893d811',
 '51c1c58b-a49f-4ac0-94ea-0c0b9bc1fe4d',
 '8ad50478-2037-

In [73]:
## lets do simsilarity search
result  = vector_store.similarity_search_with_score(
    "Malabar"
)

In [71]:
result

[(Document(id='ca47e300-43f9-44d1-932b-cb0fe99e91fe', metadata={'producer': 'ReportLab PDF Library - (opensource)', 'creator': '(unspecified)', 'creationdate': '2026-02-12T18:03:04+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2026-02-12T18:03:04+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': '../../docs/kerala.pdf', 'total_pages': 9, 'page': 6, 'page_label': '7', 'start_index': 791}, page_content='language lovers, especially on Vijayadasami when children are initiated into learning.\nMalappuram is a centre for both Hindu-Vedic learning and Islamic philosophy, and its mosques\nand temples are known for spectacular festivals. Nilambur, nestled beside the Chaliyar River,\nis famous for the world’s oldest teak plantation. The Kadalundi Bird Sanctuary, located at the\nestuary where the Kadalundi River meets the Arabian Sea, attracts migratory birds from across\nthe globe. Malappuram’s food culture is deeply rooted in Malabar tradit